In [ ]:
import sys
import pandas as pd
import numpy as np
import cufflinks as cf
sys.path.append('../../')
cf.go_offline()

import bitcoin.backtester.viewgen as vg
import bitcoin.storage.util as sutil
import bitcoin.storage.api as st
import bitcoin.util as util
import bitcoin.params as params
import bitcoin.strategies.mom as mom

In [ ]:
%load_ext snakeviz
#%load_ext line_profiler

In [ ]:
reload(st)

name = '2017-11-10_00_to_2017-11-10_03'
%time dataset = st.get_dataset(name)

In [ ]:
reload(mom)
reload(vg)

strategy = mom.MomStrategy()
viewgen = vg.ViewGen(strategy)
end = pd.datetime(2017, 11, 10, 0, 30)

%time viewgen.run(dataset)

In [ ]:
result = viewgen.result
result['view_diff'] = result['view'].diff()
result.head()

In [ ]:
group = result.groupby(pd.TimeGrouper('1T'))
agg_view = group.view.apply(np.median)
agg_price = group.price.last()
agg_result = pd.concat([agg_view, agg_price], axis=1)
agg_result['fwd_ret'] = agg_price.diff().shift(-1)
agg_result.head()

In [ ]:
agg_result.groupby(agg_result.view)['fwd_ret'].describe()

In [ ]:
result['balance'] = np.where(result.view_diff == 1,
                             -result.price, 
                             np.where(result.view_diff == -1, result.price, 0))
result.head()

In [ ]:
result[result.view_diff.abs() > 0].head()

In [ ]:
cum_balance = result.balance.cumsum()
cum_balance = cum_balance[cum_balance.abs() <= 1000]
cum_balance = cum_balance[cum_balance.diff().abs() > 0]
print cum_balance.shape
cum_balance.tail()

In [ ]:
cum_balance.iplot()

In [ ]:
result[['price', 'view']].iplot(secondary_y=['view'])

In [ ]:
bins = 10
prices = result.price

diffs = prices.diff()
groups = diffs.groupby(pd.TimeGrouper(freq='1T'))
past = groups.mean()
fwd = past.shift(-1)
past_fwd = pd.DataFrame(dict(past=past, fwd=fwd))
past_fwd = past_fwd[~past_fwd.past.isnull()]

chunks = np.array_split(past_fwd.sort_values('past'), bins)
deciles = pd.DataFrame([chunk.mean() for chunk in chunks])
deciles.iplot(kind='bar', title='TS Mom Price Differences')
deciles

In [ ]:
past_fwd.head()

In [ ]:
timestamp = pd.datetime.utcnow()
from datetime import datetime

In [ ]:
isinstance(timestamp, datetime)

In [ ]:
len("2017-11-23T09:33:51.316986Z")